# 뎅기 바이러스 백신 설계: Consensus Sequence 및 보존 영역 분석

이 노트북은 뎅기 바이러스(DENV-1, DENV-2, DENV-3, DENV-4) 각 혈청형별로 NCBI GenBank에서 다중 서열을 다운로드하고, MAFFT로 정렬하여 consensus sequence를 생성하며, 모든 혈청형 간 보존 영역을 분석하여 백신 타겟을 제안합니다.

## 기능
- **자동 다운로드**: NCBI에서 각 혈청형별 완전 게놈 서열(5-10개) 다운로드.
- **Consensus 생성**: MAFFT로 다중 서열 정렬 후 consensus 계산.
- **Pan-DENV 분석**: Shannon 엔트로피로 보존 영역 식별.
- **백신 타겟**: E, NS1, NS5 등 면역원성 높은 영역 제안.

## 요구사항
- Python 패키지: biopython, scikit-bio, pandas, numpy, matplotlib, seaborn
- MAFFT: 다중 서열 정렬을 위해 설치 (`mafft --version`로 확인)
- NCBI Entrez: 이메일 설정 필요 (다운로드 인증)

## 출력
- FASTA: 각 혈청형 consensus (`output/DENV*_consensus.fasta`)
- CSV: 엔트로피 (`pan_denv_entropy.csv`), 보존 영역 (`pan_denv_conserved_regions.csv`), 백신 타겟 (`vaccine_targets.csv`)
- PNG: 엔트로피 그래프 (`pan_denv_entropy.png`)


In [ ]:
# 라이브러리 가져오기
from Bio import SeqIO, AlignIO, Entrez
from Bio.Align.Applications import MafftCommandline
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from skbio import DNA, TabularMSA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time

# 디렉토리 설정
input_dir = "genome_sequences"
output_dir = "output"
if not os.path.exists(input_dir):
    os.makedirs(input_dir)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 혈청형 목록
serotypes = ["DENV1", "DENV2", "DENV3", "DENV4"]

# NCBI Entrez 설정 (이메일 필수)
Entrez.email = "your.email@example.com"  # 사용자의 이메일로 변경
Entrez.api_key = None  # 필요 시 NCBI API 키 추가


## 1단계: NCBI에서 서열 다운로드

각 혈청형별로 완전 게놈 서열을 NCBI GenBank에서 검색하여 FASTA 파일로 저장합니다.
- 검색어: "Dengue virus [serotype] complete genome"
- 조건: 길이 10,000-11,000bp, 최대 10개 서열
- 저장: `genome_sequences/DENV*_genome.fasta`


In [ ]:
# NCBI에서 서열 다운로드 함수
def download_sequences(serotype, max_sequences=10, min_length=10000, max_length=11000):
    print(f"{serotype} 서열 다운로드 시작...")
    query = f"Dengue virus {serotype[-1]}[Organism] complete genome"
    try:
        # 검색
        handle = Entrez.esearch(db="nucleotide", term=query, retmax=max_sequences)
        record = Entrez.read(handle)
        handle.close()
        ids = record["IdList"]
        if not ids:
            print(f"{serotype}: 검색 결과 없음.")
            return []
        
        # 서열 다운로드
        sequences = []
        for id in ids:
            try:
                handle = Entrez.efetch(db="nucleotide", id=id, rettype="fasta", retmode="text")
                seq_record = SeqIO.read(handle, "fasta")
                handle.close()
                if min_length <= len(seq_record.seq) <= max_length:
                    sequences.append(seq_record)
                else:
                    print(f"{id}: 길이 부적합 ({len(seq_record.seq)}bp)")
                time.sleep(0.5)  # NCBI 요청 제한 방지
            except Exception as e:
                print(f"{id} 다운로드 오류: {e}")
        
        # FASTA 파일 저장
        if sequences:
            output_fasta = os.path.join(input_dir, f"{serotype}_genome.fasta")
            SeqIO.write(sequences, output_fasta, "fasta")
            print(f"{serotype}: {len(sequences)}개 서열 저장 ({output_fasta})")
        return sequences
    except Exception as e:
        print(f"{serotype} 다운로드 실패: {e}")
        return []

# 각 혈청형별 서열 다운로드
sequence_data = {}
for serotype in serotypes:
    sequences = download_sequences(serotype)
    sequence_data[serotype] = sequences
    print(f"{serotype}: {len(sequences)}개 서열 다운로드 완료.")

## 2단계: 서열 검증

다운로드한 서열을 검증합니다.
- 최소 길이: 10,000bp
- 서열 수 확인
- 비-뉴클레오타이드 문자 경고


In [ ]:
# 서열 검증
min_length = 10000
for serotype in serotypes:
    sequences = sequence_data[serotype]
    print(f"{serotype}: {len(sequences)}개 서열 검증 시작.")
    for seq in sequences:
        if len(seq.seq) < min_length:
            print(f"경고: {seq.id} 길이 부족 ({len(seq.seq)}bp)")
        else:
            print(f"{seq.id}: {len(seq.seq)}bp, 유효")
        # 비-뉴클레오타이드 문자 확인
        invalid_chars = set(str(seq.seq).upper()) - set("ATGCN-")
        if invalid_chars:
            print(f"경고: {seq.id} 비-뉴클레오타이드 문자 발견: {invalid_chars}")

## 3단계: 혈청형별 Consensus Sequence 생성

각 혈청형의 다중 서열을 MAFFT로 정렬하고, 위치별 최빈 뉴클레오타이드를 계산하여 consensus sequence를 생성합니다.
- 단일 서열: 그대로 consensus로 사용
- 다중 서열: MAFFT 정렬 후 최빈값 계산
- 출력: `output/DENV*_consensus.fasta`


In [ ]:
# Consensus 생성 함수
def generate_consensus(sequences, serotype, output_fasta):
    if not sequences:
        print(f"{serotype}: 서열 없음, 스킵.")
        return None
    
    if len(sequences) == 1:
        # 단일 서열: 그대로 사용
        consensus = sequences[0]
        consensus.id = f"{serotype}_consensus"
        consensus.description = f"{serotype} Consensus (단일 서열)"
        SeqIO.write([consensus], output_fasta, "fasta")
        print(f"{serotype} Consensus 저장: {output_fasta} (단일 서열)")
        return str(consensus.seq).upper()
    
    # 다중 서열: MAFFT 정렬
    temp_fasta = os.path.join(output_dir, f"{serotype}_temp.fasta")
    SeqIO.write(sequences, temp_fasta, "fasta")
    
    # MAFFT 실행
    aligned_fasta = os.path.join(output_dir, f"{serotype}_aligned.fasta")
    try:
        mafft_cline = MafftCommandline(input=temp_fasta, auto=True)
        stdout, stderr = mafft_cline()
        with open(aligned_fasta, "w") as handle:
            handle.write(stdout)
    except Exception as e:
        print(f"{serotype} MAFFT 오류: {e}")
        os.remove(temp_fasta)
        return None
    
    # 정렬 결과 읽기
    try:
        alignment = AlignIO.read(aligned_fasta, "fasta")
    except Exception as e:
        print(f"{serotype} 정렬 읽기 오류: {e}")
        os.remove(temp_fasta)
        os.remove(aligned_fasta)
        return None
    
    # Consensus 계산
    length = alignment.get_alignment_length()
    consensus = []
    for i in range(length):
        column = [record.seq[i].upper() for record in alignment]
        counts = pd.Series(column).value_counts()
        most_common = counts.idxmax()
        if most_common in "ATGC":  # 유효 뉴클레오타이드만
            consensus.append(most_common)
        else:
            consensus.append("N")  # 갭(-) 또는 비유효 문자는 N
    
    consensus_seq = ''.join(consensus)
    consensus_record = SeqRecord(Seq(consensus_seq), id=f"{serotype}_consensus", description=f"{serotype} Consensus")
    SeqIO.write([consensus_record], output_fasta, "fasta")
    print(f"{serotype} Consensus 저장: {output_fasta}")
    
    # 임시 파일 삭제
    os.remove(temp_fasta)
    os.remove(aligned_fasta)
    return consensus_seq

# 각 혈청형별 Consensus 생성
consensus_sequences = {}
for serotype in serotypes:
    sequences = sequence_data[serotype]
    output_fasta = os.path.join(output_dir, f"{serotype}_consensus.fasta")
    consensus_seq = generate_consensus(sequences, serotype, output_fasta)
    if consensus_seq:
        consensus_sequences[serotype] = consensus_seq

## 4단계: Pan-DENV 보존 영역 분석

각 혈청형의 consensus sequence를 결합하여 MAFFT로 정렬하고, Shannon 엔트로피를 계산하여 보존 영역(엔트로피 < 0.8)을 식별합니다.
- 입력: 각 혈청형 consensus
- 출력: 엔트로피 CSV, 보존 영역 CSV


In [ ]:
# Consensus 결합
combined_fasta = os.path.join(output_dir, "pan_denv_consensus.fasta")
combined_records = []
for serotype in serotypes:
    if serotype in consensus_sequences:
        seq = consensus_sequences[serotype]
        record = SeqRecord(Seq(seq), id=serotype, description=f"{serotype} Consensus")
        combined_records.append(record)

if len(combined_records) < 2:
    print("오류: Pan-DENV 분석을 위한 Consensus 부족.")
else:
    SeqIO.write(combined_records, combined_fasta, "fasta")
    
    # MAFFT 실행
    combined_aligned_fasta = os.path.join(output_dir, "pan_denv_aligned.fasta")
    try:
        mafft_cline = MafftCommandline(input=combined_fasta, auto=True)
        stdout, stderr = mafft_cline()
        with open(combined_aligned_fasta, "w") as handle:
            handle.write(stdout)
    except Exception as e:
        print(f"Pan-DENV MAFFT 오류: {e}")
        combined_aligned_fasta = None
    
    # 엔트로피 계산
    if combined_aligned_fasta and os.path.exists(combined_aligned_fasta):
        try:
            alignment = AlignIO.read(combined_aligned_fasta, "fasta")
            sequences = [DNA(str(record.seq)) for record in alignment]
            msa = TabularMSA(sequences)
            entropy = msa.conservation(metric="inverse_shannon_uncertainty")
            
            # 엔트로피 저장
            pan_denv_df = pd.DataFrame({
                "위치": range(1, len(entropy) + 1),
                "엔트로피": entropy
            })
            pan_denv_df.to_csv(os.path.join(output_dir, "pan_denv_entropy.csv"), index=False)
            
            # 보존 영역 (엔트로피 < 0.8)
            conserved_regions = pan_denv_df[pan_denv_df["엔트로피"] < 0.8]
            conserved_regions.to_csv(os.path.join(output_dir, "pan_denv_conserved_regions.csv"), index=False)
            print("Pan-DENV 보존 영역 저장: pan_denv_conserved_regions.csv")
        except Exception as e:
            print(f"Pan-DENV 분석 오류: {e}")
        finally:
            if os.path.exists(combined_aligned_fasta):
                os.remove(combined_aligned_fasta)

## 5단계: 엔트로피 시각화

Pan-DENV 엔트로피를 그래프로 시각화하여 보존 영역을 확인합니다.
- 출력: `output/pan_denv_entropy.png`


In [ ]:
# 엔트로피 그래프
try:
    if 'pan_denv_df' in locals():
        plt.figure(figsize=(15, 6))
        sns.lineplot(x=pan_denv_df["위치"], y=pan_denv_df["엔트로피"], label="Pan-DENV")
        plt.axhline(y=0.8, color="red", linestyle="--", label="임계값 (0.8)")
        plt.xlabel("위치 (bp)")
        plt.ylabel("Shannon 엔트로피")
        plt.title("Pan-DENV 혈청형 간 보존성 분석")
        plt.legend()
        plt.savefig(os.path.join(output_dir, "pan_denv_entropy.png"))
        plt.show()
    else:
        print("엔트로피 데이터 없음: 그래프 생성 불가.")
except Exception as e:
    print(f"엔트로피 그래프 오류: {e}")

## 6단계: 백신 타겟 주석 및 제안

보존 영역을 뎅기 게놈 구조(E, NS1, NS5 등)에 매핑하고, 백신 타겟을 제안합니다.
- 출력: `pan_denv_conserved_regions_annotated.csv`, `vaccine_targets.csv`


In [ ]:
# 뎅기 게놈 구조 (대략적 위치)
genome_map = [
    {"영역": "5'UTR", "시작": 1, "끝": 100},
    {"영역": "C", "시작": 101, "끝": 450},
    {"영역": "prM", "시작": 451, "끝": 900},
    {"영역": "E", "시작": 901, "끝": 2400},
    {"영역": "NS1", "시작": 2401, "끝": 3450},
    {"영역": "NS2A", "시작": 3451, "끝": 4100},
    {"영역": "NS2B", "시작": 4101, "끝": 4500},
    {"영역": "NS3", "시작": 4501, "끝": 6300},
    {"영역": "NS4A", "시작": 6301, "끝": 6700},
    {"영역": "NS4B", "시작": 6701, "끝": 7400},
    {"영역": "NS5", "시작": 7401, "끝": 10100},
    {"영역": "3'UTR", "시작": 10101, "끝": 10700}
]

# 보존 영역 주석
try:
    if 'conserved_regions' in locals() and not conserved_regions.empty:
        def annotate_position(position):
            for region in genome_map:
                if region["시작"] <= position <= region["끝"]:
                    return region["영역"]
            return "알 수 없음"

        conserved_regions = conserved_regions.copy()
        conserved_regions["영역"] = conserved_regions["위치"].apply(annotate_position)

        # 주석 결과 저장
        conserved_regions.to_csv(os.path.join(output_dir, "pan_denv_conserved_regions_annotated.csv"), index=False)
        print("주석 포함 보존 영역 저장: pan_denv_conserved_regions_annotated.csv")
        
        # 백신 타겟 제안
        vaccine_targets = conserved_regions.groupby("영역").size().reset_index(name="보존_위치_수")
        vaccine_targets = vaccine_targets[vaccine_targets["영역"] != "알 수 없음"]
        vaccine_targets = vaccine_targets.sort_values(by="보존_위치_수", ascending=False)
        vaccine_targets.to_csv(os.path.join(output_dir, "vaccine_targets.csv"), index=False)
        print("백신 타겟 제안 저장: vaccine_targets.csv")
        print(vaccine_targets)
    else:
        print("보존 영역 데이터 없음: 주석 불가.")
except Exception as e:
    print(f"보존 영역 주석 오류: {e}")

## 요약

- **Consensus Sequence**: `output/DENV*_consensus.fasta`에 저장.
- **Pan-DENV 엔트로피**: `output/pan_denv_entropy.csv`.
- **보존 영역**: `output/pan_denv_conserved_regions.csv` 및 `pan_denv_conserved_regions_annotated.csv`.
- **백신 타겟**: `output/vaccine_targets.csv`에 제안.
- **그래프**: `output/pan_denv_entropy.png`.

## 다음 단계
- **추가 서열**: NCBI 외 데이터(GISAID 등) 추가 시 FASTA 파일 제공.
- **백신 설계**:
  - E, NS1, NS5 기반 mRNA 서열 설계.
  - IEDB로 에피토프 예측.
- **검증**: 더 많은 서열로 보존성 재확인.

## 문제 해결
- **NCBI 오류**: Entrez.email 올바르게 설정, 인터넷 연결 확인.
- **MAFFT 오류**: `mafft --version`으로 설치 확인.
- **메모리 문제**: 서열 50개 이상 시 16GB RAM 권장.
